# Relatório Parcial Projeto de Supercomputação

## Aluno: Arthur Gomes Chieppe

Algortimos implementados:
* Gulosa
* Gulosa aleatória


## Etapa 1: Geração dos arquivos de input

### 1.1 Gerador automatizado

Para a geração dos arquivos de input do algoritmo, foi utilizado o código em C++ disponibilizado na página de introdução do projeto. Nenhum alteração foi feita, e o código encontra-se em /input_generator/gerador.cpp.

Foi desenvolvido um programa em Python responsável por rodar o gerador supracitado. Dessa forma, é possível especificar o número máximo de filmes e categorias que deseja-se gerar. O programa está localizado em `/input_generator/generate_inputs.py`, sendo que o C++ compilado tem de estar na mesma pasta. Os arquivos gerados são salvos em um novo diretório, em `/input_generator/input_files/`.O modo de uso do programa pode ser acessado com a flag --help, e está replicado abaixo:

```
python3 generate_inputs.py --help
usage: generate_inputs.py [-h] [--n_movies N_MOVIES] [--n_cat N_CAT]

Generate inputs for the model

options:
  -h, --help           show this help message and exit
  --n_movies N_MOVIES  Number of movies to generate
  --n_cat N_CAT        Number of categories to generate
```


Note que o gerador gerará os filmes respeitando um intervalo de 50 filmes entre os arquivos de inputs gerados.

## Etapa 2: A Heurística Gulosa

### 2.1 Definição de Escopo e Regras dos Algortimos

Foram feitas escolhas por questões de padronização e coerência dos resultados a serem obtidos. As regras abaixo são gerais, portanto valeu tanto para a heurística gulosa quanto a aleatória.

1. Não são admitidos filmes que tem datas de início e fim em dias diferentes. (Ex: início às 23 e fim às 2)
2. Não são admitidos filmes que começam e terminam no mesmo horário, pois isso implicaria em duração igual a 0. (Ex: início às 15 e fim às 15)

Os filmes que se enquadram em qualquer uma das situações acima são considerados inválidos, portanto não podem ser adicionados ao quadro de horários.


 ### 2.2 O algoritmo

 Inicialmente, implementou-se um algortimo com abordagem gulosa para a resolução do problema. De forma simplória, a abordagem escolhida baseia-se em:

 1. Ordenar os filmes por hora de fim crescente.
 2. Percorrer a lista de filmes ordenada, verificando se cabem no quadro de horários.
 3. Verificar se há vagas disponíveis para a categoria do filme em questão

Se forem satisfeitas as condições 2 e 3, o filme é adicionado ao quadro de horários.
 

### 2.3 Implementação em C++

A implementação do algoritmo está disponível em `heuristicas/gulosa/gulosa_filmes.cpp`. A compilação pode ser feita com o comando: `g++ gulosa_files.cpp -Wall -O3 -g -o gulosa` (assumindo que o usuário encontra-se na pasta do arquivo). Usuários de Apple Silicon podem compilar com `g++-12 gulosa_files.cpp -Wall -O3 -g -o gulosa`.

### 2.3.1 Varíaveis

1. bool timetable[24]
* Array de booleanos de tamanho 24, representando as horas do dia a serem ocupadas pelos filmes.
* Todos os valores são inicados como false.
* Obs: como pode ser constatado no código fonte, um filme que dure das 10 até às 11 ocupa apenas o timetable[10], de forma que seja possível que outro filme comece às 11.

2. 